# Hub-and-Spoke Agent Framework Demo

This notebook demonstrates the architectural split between a central **Hub** and specialized **Spokes**.

### Responsibilities Split:
- **Hub**: Registry, Lifecycle, AuthN/AuthZ, Data Policy, Routing, Centralized Observability.
- **Spokes**: Domain Intelligence, Models (Ollama), Rules, Local Optimization.

## 1. Setup and Imports

In [ ]:
import sys
import os
from datetime import datetime
import json

# Ensure local modules are importable
sys.path.append(os.getcwd())

from common.models import Transaction, SpokeCapability
from hub.registry import AgentRegistry
from hub.orchestrator import Orchestrator
from hub.observability import AuditStore
from spokes.implementations import FraudSpoke, AMLSpoke

## 2. Initialize the Hub infrastructure
We initialize the registry for Spokes and the central audit store.

In [ ]:
registry = AgentRegistry()
audit_store = AuditStore(storage_path="central_audit.jsonl")
orchestrator = Orchestrator(registry, audit_store)

print("Hub Infrastructure Initialized.")

## 3. Register Spokes
Spokes register themselves with the Hub, announcing their capabilities.

In [ ]:
# Initialize Spokes
fraud_agent = FraudSpoke(name="FraudSentinel", model_name="llama3.2")
aml_agent = AMLSpoke(name="AMLGuardian", model_name="llama3.2")

# Register Spokes in the Hub Registry
registry.register_spoke("FraudSentinel", [SpokeCapability.FRAUD], "local://fraud")
registry.register_spoke("AMLGuardian", [SpokeCapability.AML], "local://aml")

registered_spokes = [fraud_agent, aml_agent]

## 4. Process a Transaction
We create a sample transaction and send it through the Hub's orchestration flow.

In [ ]:
sample_tx = Transaction(
    transaction_id="TX-998877",
    amount=15000.00,
    sender_id="CUST-123",
    receiver_id="CUST-456",
    metadata={"is_international": True, "payment_method": "wire_transfer"}
)

print(f"Processing Transaction: {sample_tx.transaction_id} for ${sample_tx.amount}")

# Step 1: Request received and orchestrated by Hub
req_id = orchestrator.process_request(sample_tx)
print(f"Request ID: {req_id}")

# Step 2: Hub routes to Spokes (in this demo, we call the registered instances)
findings = orchestrator.execute_spokes(req_id, sample_tx, registered_spokes)

for finding in findings:
    print(f"\n[Spoke: {finding.spoke_name}] Findings:")
    print(f" - Severity: {finding.severity}")
    print(f" - Score: {finding.score}")
    print(f" - Rationale: {finding.rationale}")

## 5. Aggregation and Final Decision
The Hub collects findings and applies a global policy to reach a consensus.

In [ ]:
final_response = orchestrator.aggregate_results(req_id, findings)

print(f"Final Decision: {final_response.final_decision}")
print(f"Consensus Risk Score: {final_response.consensus_score}")

## 6. Accessing Centralized Audit Logs
The Hub maintains the gold standard for what happened across all agents.

In [ ]:
print("Recent Audit Logs:")
logs = audit_store.get_logs()
for log in logs[-10:]: # Show last 10 events
    print(f"[{log['timestamp']}] {log['event_type']} - {json.dumps(log['data'])[:100]}...")